<a href="https://colab.research.google.com/github/FatemehPasban/ChatBot/blob/main/Copy_of_multilabel__joint_intent_classification_slot_filling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install -q transformers

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 3.2MB 21.2MB/s 
     |████████████████████████████████| 890kB 38.4MB/s 


In [ ]:
# Import required packages

import numpy as np
import pandas as pd
import os
from os.path import join

from sklearn.model_selection import train_test_split
#import tensorflow

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch, gc 
from transformers import BertConfig, BertTokenizer,BertModel,AutoTokenizer

In [ ]:
ROOT_PATH='/content/drive/MyDrive/AI_BOT/network/final/'

slot_path2 = join(ROOT_PATH, 'slot_names2.txt')
slot_path = join(ROOT_PATH, 'slot_names.txt')
data_path = join(ROOT_PATH, 'AI_BOT_dataset 2.xls') 

MODEL_NAME_OR_PATH = "HooshvareLab/bert-fa-base-uncased"
#tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
raw_data = pd.read_excel(data_path)
raw_data

,h1,h2,h3,h4,h5,h6,h7,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24
0,ساعت جهانی,<=>,در,نیویورک,ساعت,چند,است,؟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,clock,o,o,b-country1,o,o,o,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ساعت جهانی,<=>,ساعت,در,ابوطبی,چند,است,؟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,clock,o,o,o,b-city1,o,o,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ساعت جهانی,<=>,الان,در,تگزاس,چه,زمانی,از,شبانه,روز,است,؟,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
449,combine-data,dates and events,weather,o,o,o,o,o,b-event1,i-event1,i-event1,o,b-year1,i-year1,b-calender1,o,o,o,NaN,NaN,NaN,NaN,NaN,NaN,NaN
450,دیتا ترکیبی,اوقات شرعی,آب و هوا,<=>,اختلاف,دمای,هوا,رم,هنگام,اذان,صبح,و,اذان,مغرب,روز,5,اذر,سال,1321,?,NaN,NaN,NaN,NaN,NaN
451,combine-data,relegues-time,weather,o,b-compare1,i-compare1,o,b-city1,o,b-time1,i-time1,o,b-time2,i-time2,b-day1,i-day1,b-month1,b-year1,i-year1,o,NaN,NaN,NaN,NaN,NaN
452,دیتا ترکیبی,اوقات شرعی,تاریخ و مناسبت ها,<=>,اختلاف,ساعت,اذان,مغرب,روز,پزشک,در,سال,2018,و,اذان,صبح,روز,جهانی,درخت کاری,در,سال,2014,?,NaN,NaN


In [ ]:
intent_list = ['آب و هوا' , 'اوقات شرعی' , 'تاریخ و مناسبت ها' , 'ساعت جهانی' , 'نامرتبط' ]

def parse_line(l):
    intent_label =[0,0,0,0,0]
    l=l.values
    s=list(l[0])
    l=list(l[1])
    sentlist = [x for x in s if str(x) != 'nan']
    labellist = [x for x in l if str(x) != 'nan']
    sentlist = [str(x) for x in sentlist]

    if sentlist[3] == '<=>':
      ind1 = intent_list.index(sentlist[1])
      ind2 = intent_list.index(sentlist[2])
      intent_label[ind1] , intent_label[ind2] = 1 , 1
      #print(intent_label, sentlist[1],sentlist[2])
      return {
              "intent_label": intent_label,
              "words": ' '.join(e for e in sentlist[4:]),
              "word_labels":' '.join(e for e in labellist[4:]),
              "length": len(sentlist)-4,
           }
    else :
      ind = intent_list.index(sentlist[0])
      intent_label[ind] = 1
      return {
              "intent_label": intent_label,
              "words": ' '.join(e for e in sentlist[2:]),
              "word_labels":' '.join(e for e in labellist[2:]),
              "length": len(sentlist)-2,
           }

In [ ]:
parse_line(raw_data[302:304])
#raw_data[295:297]

{'intent_label': [0, 0, 0, 1, 0],
 'length': 6,
 'word_labels': 'o o b-city1 o o o',
 'words': 'ساعت در ابوطبی چند است ؟'}

In [ ]:
parsed = [parse_line(raw_data[i:i+2]) for i in range(0,len(raw_data),2)]
df_data = pd.DataFrame([p for p in parsed if p is not None])

df_data

,intent_label,words,word_labels,length
0,"[0, 0, 0, 1, 0]",در نیویورک ساعت چند است ؟,o b-country1 o o o o,6
1,"[0, 0, 0, 1, 0]",ساعت در ابوطبی چند است ؟,o o b-city1 o o o,6
2,"[0, 0, 0, 1, 0]",الان در تگزاس چه زمانی از شبانه روز است ؟,b-time1 o b-state1 o o o o o o o,10
3,"[0, 0, 0, 1, 0]",الان در مسکو ساعت دقیقاً چند است ؟,b-time1 o b-city1 o o o o o,8
4,"[0, 0, 0, 1, 0]",در تاشکند این لحظه چه زمانی از روز است ؟,o b-city1 b-time1 i-time1 o o o o o o,10
...,...,...,...,...
222,"[1, 1, 0, 0, 0]",دمای هوا هنگام اذان ظهر در روز 11 اسفند 1369 ...,o o o b-time1 i-time1 o b-day1 i-day1 b-month1...,14
223,"[1, 0, 1, 0, 0]",در کریسمس سال 1367 شمسی دمای هوا چند درجه سلسی...,o b-event1 b-year1 i-year1 b-calender1 o o o b...,12
224,"[1, 0, 1, 0, 0]",اب و هوا در روز جهانی زن در سال 1390 شمسی چگون...,o o o o b-event1 i-event1 i-event1 o b-year1 i...,14
225,"[1, 1, 0, 0, 0]",اختلاف دمای هوا رم هنگام اذان صبح و اذان مغر...,b-compare1 i-compare1 o b-city1 o b-time1 i-ti...,16


In [ ]:
labels = intent_list
intent_map={'آب و هوا': 0, 'اوقات شرعی': 1, 'تاریخ و مناسبت ها': 2, 'ساعت جهانی': 3,'نامرتبط':4}
label2id = intent_map
id2label = {v: k for k, v in intent_map.items()}

file = open(slot_path,'r')
intents = file.readlines()
slot_names=["[PAD]"]
for item in intents:
  slot_names.append(item.replace('\n',''))
slot_map = {}
for label in slot_names:
    slot_map[label] = len(slot_map)

id2label_slot = {v: k for k, v in slot_map.items()}
id2label_slot

{0: '[PAD]',
 1: 'b-date1',
 2: 'i-date1',
 3: 'b-date2',
 4: 'i-date2',
 5: 'b-time1',
 6: 'i-time1',
 7: 'b-time2',
 8: 'i-time2',
 9: 'b-day1',
 10: 'i-day1',
 11: 'b-day2',
 12: 'i-day2',
 13: 'b-week1',
 14: 'i-week1',
 15: 'b-month1',
 16: 'i-month1',
 17: 'b-year1',
 18: 'i-year1',
 19: 'b-year2',
 20: 'i-year2',
 21: 'b-loc1',
 22: 'i-loc1',
 23: 'b-loc2',
 24: 'i-loc2',
 25: 'b-city1',
 26: 'i-city1',
 27: 'b-city2',
 28: 'i-city2',
 29: 'b-state1',
 30: 'i-state1',
 31: 'b-state2',
 32: 'i-state2',
 33: 'b-country1',
 34: 'i-country1',
 35: 'b-country2',
 36: 'i-country2',
 37: 'b-miss1',
 38: 'i-miss1',
 39: 'b-miss2',
 40: 'i-miss2',
 41: 'b-standard1',
 42: 'i-standard1',
 43: 'b-standard2',
 44: 'i-standard2',
 45: 'b-compare1',
 46: 'i-compare1',
 47: 'b-degree1',
 48: 'i-degree1',
 49: 'b-description1',
 50: 'i-description1',
 51: 'b-event1',
 52: 'i-event1',
 53: 'b-event2',
 54: 'i-event2',
 55: 'b-counter1',
 56: 'i-counter1',
 57: 'b-count-devise1',
 58: 'i-count-de

#Split data

In [ ]:
train, test = train_test_split(df_data, test_size=0.08, random_state=3)

df_train = train.reset_index(drop=True)
df_test = test.reset_index(drop=True)
df_test

,intent_label,words,word_labels,length
0,"[0, 0, 1, 0, 0]",روز جهانی پزشک به قمری در سال ۱۴۰۵ ؟,b-event1 i-event1 i-event1 o b-calender1 o b-y...,9
1,"[0, 1, 0, 0, 0]",ساعت اذان ظهر بیروت ؟,b-time1 i-time1 i-time1 b-city1 o,5
2,"[0, 0, 0, 1, 0]",ساعت در تورنتو چند است ؟,o o b-city1 o o o,6
3,"[0, 0, 0, 1, 0]",الان در مسکو ساعت دقیقاً چند است ؟,b-time1 o b-city1 o o o o o,8
4,"[0, 0, 0, 1, 0]",اختلاف ساعت بین پایتخت ایران و پاریس چقدر است...,b-compare1 o o o b-country1 o b-city1 o o o,11
5,"[1, 0, 1, 0, 0]",دمای هوا در روز عاشورا چند درجه بوده ?,o o o b-event1 i-event1 o o o o,9
6,"[1, 0, 0, 0, 0]",امروز میانگین دمای هوای مشهد سردتر است یا تهران ؟,b-day1 b-standard1 o o b-city1 b-compare1 o b-...,10
7,"[0, 1, 1, 0, 0]",نماز صبح در سالگرد روز زن در سال 1332 چه ساع...,b-time1 i-time1 o b-event1 i-event1 i-event1 o...,14
8,"[0, 0, 0, 0, 1]",زیستگاه اصلی کوالا کدام کشور است ؟,o o o o o o,7
9,"[0, 0, 0, 1, 0]",الان به وقت پایتخت ایران چه زمانی از روز است ؟,b-time1 o o o b-country1 o o o o o o,11


# Configuration

In [ ]:
# general config

train_sequence_lengths = [len(tokenizer.encode(text))
                          for text in df_data["words"]]
max_length = max(train_sequence_lengths)
MAX_LEN = max_length+4
MAX_LEN

27

In [ ]:
# setup the tokenizer and configuration

tokenizer = BertTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
config = BertConfig.from_pretrained(
    MODEL_NAME_OR_PATH)

#print(config)

## Join Intent Classification and Slot Filling

The following function generates token-aligned integer labels from the BIO word-level annotations. In particular, if a specific word is too long to be represented as a single token, we expand its label for all the tokens of that word while taking care of using "B-" labels only for the first token and then use "I-" for the matching slot type for subsequent tokens of the same word:

In [ ]:
def encode_token_labels(text_sequences, slot_names, tokenizer, slot_map,
                        max_length):
    encoded = np.zeros(shape=(len(text_sequences), max_length), dtype=np.int32)
    for i, (text_sequence, word_labels) in enumerate(
            zip(text_sequences, slot_names)):
        encoded_labels = []
        for word, word_label in zip(text_sequence.split(), word_labels.split()):
            tokens = tokenizer.tokenize(word)
            encoded_labels.append(slot_map[word_label])
            expand_label = word_label.replace("B-", "I-")
            if not expand_label in slot_map:
                expand_label = word_label
            encoded_labels.extend([slot_map[expand_label]] * (len(tokens) - 1))
        encoded[i, 1:len(encoded_labels) + 1] = encoded_labels
    return encoded


slot_train = encode_token_labels(
    df_train["words"], df_train["word_labels"], tokenizer, slot_map, MAX_LEN)
slot_test = encode_token_labels(
    df_test["words"], df_test["word_labels"], tokenizer, slot_map, MAX_LEN)

slot_test[0]

array([ 0,  5,  6, 61,  9, 15, 61, 61, 61,  7,  7, 61, 51, 61, 17, 17, 45,
       61, 61,  0,  0,  0,  0,  0,  0,  0,  0], dtype=int32)

# Input Embeddings

### Dataset

In [ ]:
class CreateDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset. """
 
    def __init__(self, tokenizer, comments, targets=None,slots_targets=None , label_list=None, max_len=MAX_LEN):
        self.comments = comments
        self.targets = targets
        self.slots_targets = slots_targets
        self.has_target = isinstance(targets, list) or isinstance(targets, np.ndarray)
 
        self.tokenizer = tokenizer
        self.max_len = max_len
        
        self.label_map = {label: i for i, label in enumerate(label_list)} if isinstance(label_list, list) else {}
        #self.slots_map = slots_map
 
    def __len__(self):
        return len(self.comments)
 
    def __getitem__(self, item):
        comment = str(self.comments[item])
 
 
        #if self.targets != None:
        target = self.targets[item]
        slots_target = self.slots_targets[item]
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }
 
        #if self.targets != None:
        inputs['targets'] = torch.tensor(target, dtype=torch.long)
        inputs['slot_targets'] = torch.tensor(slots_target,dtype=torch.long)
        return inputs
 
 
def create_data_loader(x, y,slots, tokenizer, max_len, batch_size, label_list):
    dataset = CreateDataset(
        comments=x,
        targets=y,
        slots_targets=slots,
        tokenizer=tokenizer,
        max_len=max_len, 
        label_list=label_list,
        )
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
#label_list = ['آب و هوا','اوقات شرعی','تاریخ و مناسبت ها','ساعت جهانی']
print(type(df_train['intent_label'].to_numpy()),type(df_train['intent_label']))
train_data_loader = create_data_loader(df_train['words'].to_numpy(), df_train['intent_label'].to_numpy(),slot_train,tokenizer, MAX_LEN, len(df_train), intent_list)
test_data_loader = create_data_loader(test['words'].to_numpy(), df_train['intent_label'].to_numpy(),slot_train, tokenizer, MAX_LEN, len(df_test), intent_list)
print(type(test_data_loader),test_data_loader)

train_data_loader2 = next(iter(train_data_loader))
test_data_loader2 = next(iter(test_data_loader))

<class 'numpy.ndarray'> <class 'pandas.core.series.Series'>
<class 'torch.utils.data.dataloader.DataLoader'> <torch.utils.data.dataloader.DataLoader object at 0x7f0a90169590>


## Model

In [ ]:
class IntentClassifierModel(nn.Module):
    

    def __init__(self, config):
        super(IntentClassifierModel, self).__init__()

        self.bert = BertModel.from_pretrained(MODEL_NAME_OR_PATH,return_dict=False)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.intent_classifier = nn.Linear(config.hidden_size, 5 )
        self.slot_classifier = nn.Linear(config.hidden_size,len(id2label_slot))
    
    def forward(self, input_ids, attention_mask, token_type_ids):
        sequence_output , pooled_output = self.bert(
            input_ids=input_ids, 
            attention_mask=attention_mask, 
            token_type_ids=token_type_ids)
        
        #print(type(sequence),sequence,type(pooled_output),pooled_output)
        sequence_output = self.dropout(sequence_output)
        slot_logits = self.slot_classifier(sequence_output)

        pooled_output = self.dropout(pooled_output)
        intent_logits = self.intent_classifier(pooled_output)

        return intent_logits, slot_logits

In [ ]:
pt_model = IntentClassifierModel(config=config)

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion2 = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(params=pt_model.parameters(), lr=3e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)

Input: (N,C) where C = number of classes
Target: (N) where each value is 0 <= targets[i] <= C-1
Output: scalar. If reduce is False, then (N) instead.

In [ ]:
#PLEASE RUN THIS CELL FOR 3 TIME
n_epochs = 50

for epoch in range(n_epochs):
  pt_model.train()
  optimizer.zero_grad()
  output_intent,output_slot = pt_model(train_data_loader2['input_ids'], train_data_loader2['attention_mask'], train_data_loader2['token_type_ids'])
  print('---------')
  print(output_intent.shape,train_data_loader2['targets'].shape)
  print(output_slot.shape,train_data_loader2['slot_targets'].shape)

  intent_target = train_data_loader2['targets'].type_as(output_intent)
  loss1 = criterion2(output_intent, intent_target)


  loss2 = 0
  for i in range(output_slot.shape[0]):
    if df_train['intent_label'][i][4]!=1:
      n = df_train['length'][i]
      l = criterion(output_slot[i,:n,:], train_data_loader2['slot_targets'][i][:n])
      loss2 = loss2 + l
  #for i in range(output_slot.shape[0]):
  #  n = df_train['length'][i]
  #  #slot_target =  train_data_loader2['slot_targets'][i][:n].type_as(output_slot)
  #  l = criterion(output_slot[i,:n,:], slot_target)
  # loss2 = loss2 + l
  #slot_targets = torch.reshape(train_data_loader2['slot_targets'], (-1,))
  #output_slot = torch.reshape(output_slot,(len(df_train)*MAX_LEN ,len(id2label_slot) ))
  #loss2 = criterion(output_slot, slot_targets)

  loss = loss1 + loss2

  loss.backward()
  optimizer.step()
  print(f"Epoch {epoch+1}/{n_epochs}, loss = {loss.item()}")

---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 1/50, loss = 91.02772521972656
---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 2/50, loss = 34.031219482421875
---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 3/50, loss = 24.031173706054688
---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 4/50, loss = 14.05322265625
---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 5/50, loss = 11.623032569885254
---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 6/50, loss = 13.55824089050293
---------
torch.Size([208, 5]) torch.Size([208, 5])
torch.Size([208, 27, 62]) torch.Size([208, 27])
Epoch 7/50, loss = 11.345762252807617
---------
torch.Size([208, 5]) torch.Siz

# prediction

In [ ]:
from torch import tensor
import numpy

intent_logits ,slot_logits = pt_model(test_data_loader2['input_ids'], test_data_loader2['attention_mask'], test_data_loader2['token_type_ids'])
_, preds_slot = torch.max(slot_logits, dim=2)
slots = preds_slot.numpy().tolist()
comments = list(test_data_loader2['comment'])
print(len(comments))
intent_logits = intent_logits.detach().numpy()
print(intent_logits)
outputs = np.array(intent_logits) >= 0.008

for i in range(len(outputs)):
  if (True in outputs[i]):
    ind=[]
    for j in range(len(outputs[i])):
      if outputs[i][j]==True:
        ind.append(j)
    print(ind)
    print('==================\n')
    print(comments[i])
    inte=[id2label[k] for k in ind] 
    print("## Intent:", inte)
    print("## Slots:")
    slott = slots[i]
    tokens = tokenizer.tokenize(comments[i])
    j=1
    for token in tokens:
      print(token ,':',id2label_slot[slott[j]])
      j+=1
  else:
    print('==================\n')
    print("## Slots: cant find :( ")



19
[[ 0.09553219  0.11387461 -0.38471997 -3.1755013  -3.0838578 ]
 [-1.1407945  -3.021513   -2.7832472   0.02433044 -2.7817984 ]
 [-2.4295554   1.7576038  -2.603657   -2.5370724  -3.8434646 ]
 [-2.5831268  -3.1802373   0.1827153  -1.2232603  -1.6886711 ]
 [ 0.27994102 -2.2700958  -3.6429088  -1.0912741  -3.2120795 ]
 [-2.5758224   1.4893578  -2.7388468  -2.4400206  -3.1937723 ]
 [-0.05572292 -2.0813663  -1.9489181  -2.0874414  -1.5263604 ]
 [-1.0565772  -1.6181002  -2.7590451  -1.9850379  -1.7074416 ]
 [ 2.3449798  -2.9500895  -3.0714471  -2.2864912  -2.7827342 ]
 [ 2.491691   -2.2151303  -2.597299   -2.5239356  -2.852442  ]
 [-0.6710018  -3.0601993  -2.5603836  -1.4409858  -1.4098389 ]
 [-1.1137457  -3.4080462  -3.2659535   0.66670156 -2.2422688 ]
 [-3.4802375  -2.00875    -3.2642941   1.3721273  -2.4546387 ]
 [-2.2002604   1.8192701  -2.5794587  -2.5845273  -3.1618173 ]
 [-1.812651    1.0496744  -3.4279914  -1.1437522  -3.6861875 ]
 [-3.398077   -1.459047   -2.8502433   1.5844426  -2

#one sample test

In [ ]:
class CreateTestDataset(torch.utils.data.Dataset):
    """ Create a PyTorch dataset. """
 
    def __init__(self, tokenizer, comments,max_len=MAX_LEN):
        self.comments = comments 
        self.tokenizer = tokenizer
        self.max_len = max_len
         
    def __len__(self):
        return len(self.comments)
 
    def __getitem__(self, item):
        comment = str(self.comments[item])
 
        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            truncation=True,
            max_length=self.max_len,
            return_token_type_ids=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt')
        
        inputs = {
            'comment': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'token_type_ids': encoding['token_type_ids'].flatten(),
        }

        return inputs
 
 
def create_Test_data_loader(x, tokenizer, max_len, batch_size):
    dataset = CreateTestDataset(
        comments=x,
        tokenizer=tokenizer,
        max_len=max_len, 
        )
    
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size)

In [ ]:
strr="اختلاف ساعت بین پایتخت کره جنوبی و کیف اوکراین چقدر است ؟"
sample=create_Test_data_loader(np.array([strr]), tokenizer, MAX_LEN, 1)
sample= next(iter(sample))
sample

{'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]]),
 'comment': ['اختلاف ساعت بین پایتخت کره جنوبی و کیف اوکراین چقدر است ؟'],
 'input_ids': tensor([[    2,  5086,  3551,  3016,  5566,  4958,  4435,  1379,  4306, 10864,
           6614,  2806,  1350,     4,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0]]),
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
          0, 0, 0]])}

In [ ]:
intent_logits ,slot_logits = pt_model(sample['input_ids'], sample['attention_mask'], sample['token_type_ids'])
intent_logits 

tensor([[-0.4820, -0.7146, -0.6822,  0.4533, -0.2373]],
       grad_fn=<AddmmBackward>)

In [ ]:
from torch import tensor
import numpy 
id2label= {0: 'آب و هوا',
            1: 'اوقات شرعی',
            2: 'تاریخ و مناسبت ها',
            3: 'ساعت جهانی',
            4: 'نامرتبط'}

id2label_slot = {0: '[PAD]',1: 'b_date1', 2: 'i_date1', 3: 'b_date2', 4: 'i_date2', 5: 'b_time1', 6: 'i_time1', 7: 'b_time2', 8: 'i_time2', 9: 'b_day1', 10: 'i_day1', 11: 'b_day2', 12: 'i_day2', 13: 'b_week1', 14: 'i_week1', 15: 'b_month1', 16: 'i_month1',17: 'b_year1',18: 'i_year1', 19: 'b_year2', 20: 'i_year2', 21: 'b_loc1', 22: 'i_loc1', 23: 'b_loc2', 24: 'i_loc2', 25: 'b_city1', 26: 'i_city1', 27: 'b_city2', 28: 'i_city2', 29: 'b_state1', 30: 'i_state1', 31: 'b_state2', 32: 'i_state2', 33: 'b_country1', 34: 'i_country1', 35: 'b_country2', 36: 'i_country2', 37: 'b_miss1', 38: 'i_miss1', 39: 'b_miss2', 40: 'i_miss2', 41: 'b_standard1', 42: 'i_standard1', 43: 'b_standard2', 44: 'i_standard2', 45: 'b_compare1', 46: 'i_compare1', 47: 'b_degree1', 48: 'i_degree1', 49: 'b_description1', 50: 'i_description1', 51: 'b_event1', 52: 'i_event1', 53: 'b_event2', 54: 'i_event2', 55: 'b_counter1', 56: 'i_counter1', 57: 'b_count-devise1', 58: 'i_count-devise1', 59: 'b_calender1', 60: 'i_calender1', 61: 'o'}

def creatDict(intent_pred , slot_pred ,Text,tokenizer):
    _, preds_slot = torch.max(slot_logits, dim=2)
    slots = preds_slot.numpy().tolist()
    intent_pred = intent_pred.detach().numpy()
    outputs = numpy.array(intent_pred) >= 0.08
    dic={}
    print(outputs[0])
    for i in range(len(outputs)):
        if (True in outputs[i]):
            ind=[]
            
            for j in range(len(outputs[i])):
                if outputs[i][j]==True:
                    ind.append(j)
            inte=[id2label[k] for k in ind] 
            dic['Intent']=inte
            slott = slots[i]
            tokens = tokenizer.tokenize(Text)
            j=1
            slot_dic={}
            ind_slots=[]
            l=0
            for token in tokens:
                slot_dic[join(token+'_'+str(l))]=id2label_slot[slott[j]]
                l+=1
                j+=1
            dic['Slots']=slot_dic
        else:
            print("## Slots: cant find :( ")
        return dic
rr=creatDict(intent_logits ,slot_logits ,strr,tokenizer)
rr

[False False False  True False]


{'Intent': ['ساعت جهانی'],
 'Slots': {'؟_11': 'o',
  'اختلاف_0': 'o',
  'است_10': 'o',
  'اوکراین_8': 'o',
  'بین_2': 'o',
  'جنوبی_5': 'o',
  'ساعت_1': 'o',
  'و_6': 'o',
  'پایتخت_3': 'o',
  'چقدر_9': 'o',
  'کره_4': 'o',
  'کیف_7': 'o'}}

# save model

In [ ]:
import pickle

model_save_name = 'MultilabelJointModel.pt'
path = F"/content/drive/MyDrive/AI_BOT/network/final/{model_save_name}" 
torch.save(pt_model, path)

In [ ]:
#import pickle
#model_save_name = 'MultilabelJointModel.pt'
#path = F"/content/drive/MyDrive/AI_BOT/network/final/{model_save_name}" 
#torch.save(pt_model, path, pickle_module=pickle, pickle_protocol=4)

#اینجا مشکل نداره لودش!!!!

In [ ]:
Model = torch.load('/content/drive/MyDrive/AI_BOT/network/final/MultilabelJointModel.pt')
Model.eval()

IntentClassifierModel(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(100000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af